In [ ]:
from notebook_prelude import *

In [ ]:
X, Y = dataset_helper.get_dataset('nyt_small', use_cached=False)
collections.Counter(Y)

In [ ]:
NUM_PER_CLASS = 25

per_class = collections.defaultdict(list)
for x, y in zip(X, Y):
    per_class[y].append(x)
per_class_count = {k: len(v) for k, v in per_class.items()}

out = {}
for y, xs in per_class.items():
    out[y] = np.random.choice(xs, size=NUM_PER_CLASS)

assert np.all([len(xs) == NUM_PER_CLASS for xs in out.values()])

In [ ]:
X, Y = [], []
for y, xs in out.items():
    Y += [y] * len(xs)
    X += xs.tolist()
assert len(X) == len(Y)

In [ ]:
X_u, Y_u = dataset_helper.get_dataset('ted_talks_unclassified')
X, Y = dataset_helper.get_dataset('ted_talks')

In [ ]:
Y[0], X[0]

In [ ]:
X_u_cmap, Y_u_cmap = dataset_helper.get_dataset_cached(dataset_helper.get_all_cached_graph_datasets(dataset_name='ted_talks_unclassified', graph_type=TYPE_CONCEPT_MAP)[0])

In [ ]:
len(Y_u_cmap)

In [ ]:
X, Y = dataset_helper.get_concept_map_combined_dataset_for_dataset('ted_talks')
X[0]

In [ ]:
from data.datasets.ted_talks import dataset as ted_talks

df = ted_talks.get_df()
url_2_label = {url: df_.label for url, df_ in df.iterrows()}

url__2_label = {url: idx for idx, url in enumerate(Y_u_cmap)}

c = collections.Counter()
X_cmap_new, Y_cmap_new = [], []
for url, df_ in df.iterrows():
    if url not in url__2_label:
        print('No graph found for:', url, df_.transcript)
        continue
    graph, _ = X_u_cmap[url__2_label[url]]
    other_url = Y_u_cmap[url__2_label[url]]
    assert other_url == url
    label = df_.label
    X_cmap_new.append((graph, str(c[label]).zfill(4)))
    Y_cmap_new.append(label)
    c[label] += 1
    
with open('data/CACHE/dataset_graph_concept_map_ted_talks-v3.npy', 'wb') as f:
    pickle.dump((X_cmap_new, Y_cmap_new), f)

In [ ]:
X_cmap, Y_cmap = dataset_helper.get_dataset_cached(dataset_helper.get_all_cached_graph_datasets(dataset_name='ted_talks', graph_type=TYPE_CONCEPT_MAP)[0])
X_cmap[0]

In [ ]:
for file in dataset_helper.get_all_cached_graph_datasets(graph_type=TYPE_COOCCURRENCE):
    print(file)
    X, Y = dataset_helper.get_dataset_cached(file)
    print(X[0].nodes())
    break

In [ ]:
X, Y = dataset_helper.get_dataset('ng20')

X = list(map(preprocessing.preprocess, X))
t = sklearn.feature_extraction.text.CountVectorizer()
t.fit(X)

In [ ]:
all_words = list(t.vocabulary_.keys())

words_len_2 = [x for x in all_words if len(x) <= 3]
for word in words_len_2:
    assert word in t.vocabulary_
    
words_len_2

In [ ]:
from itertools import chain

all_text = list(chain.from_iterable(X))
all_text = ''.join(all_text)
word_counts_ = t.transform([all_text])
word_count_indices = word_counts_.nonzero()[1]

In [ ]:
total_words = word_counts_.sum()
word_counts = {}
idx_2_word = {idx: word for word, idx in t.vocabulary_.items()}
print(total_words)
for idx in word_count_indices:
    word = idx_2_word[idx]
    count = word_counts_[0, idx]
    frequency = count / total_words
    print(word, count)
    break

In [ ]:
all_words_ = all_text.lower().split()
word_counter = collections.Counter(all_words_)

In [ ]:
all_word_counts = list(word_counter.values())
s = pd.Series(all_word_counts)
#.plot(kind='hist', bins=300)
s[(s > s.quantile(0.01)) & (s < s.quantile(0.99))].plot(kind='hist', bins=40)

In [ ]:
len(s[s == 1]) / len(s)

In [ ]:
for file in results_helper.get_result_filenames_from_folder():
    if 'result___ling-spam__graph_gram__dataset_graph_cooccurrence_1_only-nouns_un-lemmatized_ling-spam.spgk-1.gram.npy' not in file: continue
    with open(file, 'rb') as f:
        res = pickle.load(f)
    pprint(res['results']['param_classifier'][0].dual_coef_.shape)

In [ ]:
GRAPHS = {}
cmaps = dataset_helper.get_all_cached_graph_datasets(graph_type=TYPE_CONCEPT_MAP)
for cache_file in cmaps:
    dataset = filename_utils.get_dataset_from_filename(cache_file)
    X, Y = dataset_helper.get_dataset_cached(cache_file=cache_file)
    X_g = graph_helper.get_graphs_only(X)
    GRAPHS[dataset] = (X_g, Y)


In [ ]:
cmap = dataset_helper.get_all_cached_graph_datasets(dataset_name='ted_talks', graph_type=TYPE_CONCEPT_MAP)[0]
X_, Y = dataset_helper.get_dataset_cached(cmap)

In [ ]:
figsize=(13, 13)

X = graph_helper.get_graphs_only(X_)
candidates = [idx for idx, g in enumerate(X) if len(g.nodes()) < 15 and len(g.nodes()) > 5]
idx = np.random.choice(candidates)
g = X[idx]
fig, ax = plt.subplots(figsize=figsize)
pos = nx.layout.circular_layout(G=g)#, iterations=100)
nx.draw_networkx(g, node_size=4, pos=pos, ax = ax)
edges = {(s, t): data['name'] for s, t, data in g.edges(data=True)}
nx.draw_networkx_edge_labels(g, pos=pos, edge_labels=edges)
cleanup_axes(ax)

In [ ]:
import experiments
from experiments import experiment_helper
experiment_helper.save_experiment_params_as_experiment_config()
print('\n'.join([filename_utils.get_dataset_from_filename(x) for x in dataset_helper.get_all_cached_graph_datasets(graph_type=TYPE_CONCEPT_MAP)]))


In [ ]:
X, Y = dataset_helper.get_dataset('imsdb')
word_counts = [len(x.split()) for x in X]

In [ ]:
pd.DataFrame(word_counts, columns = ['word_counts']).plot(kind='hist', bins=120)
len(word_counts)

In [ ]:
FIG_SIZE = (30, 30)
#FIG_SIZE = (5, 5)
g = nx.read_gml('tmp/concept-graph.graph')

node_mapping = {}
for node, data in g.nodes(data=True):
    node_mapping[node] = data['name']
node_labels = list(node_mapping.values())
assert len(node_labels) == len(set(node_labels))


nx.relabel_nodes(g, node_mapping, copy=False)
edges = g.edges(data=True)
edge_labels = {(k, v): data['name'] for k, v, data in edges}

for layout in [nx.layout.circular_layout, nx.layout.fruchterman_reingold_layout, nx.layout.random_layout, nx.layout.shell_layout, nx.layout.spectral_layout, nx.layout.spring_layout]:
    fig, ax = plt.subplots(figsize=FIG_SIZE)
    pos = layout(g)
    nx.draw_networkx(g, ax=ax, pos=pos, node_size=10)
    nx.draw_networkx_edge_labels(g, pos, edge_labels=edge_labels)
    ax.grid(False)
    ax.set_title(layout.__name__)
    fig.tight_layout()
    save_fig(fig, 'concept_map_{}'.format(layout.__name__), folder='tmp/concept_map_example')